# Multivariate Poisson AR

In [1]:
using GLMCopula, Random, Statistics, Test, LinearAlgebra, StatsFuns, GLM

In [2]:
using GLMCopula, DelimitedFiles, LinearAlgebra, Random, GLM, MixedModels, CategoricalArrays
using Random, Roots, SpecialFunctions
using DataFrames, DelimitedFiles, Statistics, ToeplitzMatrices
import StatsBase: sem

p = 3    # number of fixed effects, including intercept

# true parameter values
Random.seed!(12345)
βtrue = rand(Uniform(-2, 2), p)
σ2true = [0.5]
ρtrue = [0.5]
trueparams = [βtrue; ρtrue; σ2true]

function get_V(ρ, n)
    vec = zeros(n)
    vec[1] = 1.0
    for i in 2:n
        vec[i] = vec[i - 1] * ρ
    end
    V = ToeplitzMatrices.SymmetricToeplitz(vec)
    V
end

#simulation parameters
samplesize = 10_000

st = time()
currentind = 1
d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64

gcs = Vector{GLMCopulaARObs{T, D, Link}}(undef, samplesize)

ni = 5# number of observations per individual
V = get_V(ρtrue[1], ni)

# true Gamma
Γ = σ2true[1] * V

# for reproducibility I will simulate all the design matrices here
Random.seed!(12345)
X_samplesize = [randn(ni, p - 1) for i in 1:samplesize]

for i in 1:samplesize
    X = [ones(ni) X_samplesize[i]]
    η = X * βtrue
    μ = exp.(η)
    vecd = Vector{DiscreteUnivariateDistribution}(undef, ni)
    for i in 1:ni
        vecd[i] = Poisson(μ[i])
    end
    nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
    # simuate single vector y
    y = Vector{Float64}(undef, ni)
    res = Vector{Float64}(undef, ni)
    rand(nonmixed_multivariate_dist, y, res)
    # X = ones(ni, 1)
    # y = Float64.(Y_nsample[i])
    V = [ones(ni, ni)]
    gcs[i] = GLMCopulaARObs(y, X, d, link)
end

# form model
gcm = GLMCopulaARModel(gcs);

In [3]:
initialize_model!(gcm);

initializing β using Newton's Algorithm under Independence Assumption
initializing variance components using MM-Algorithm
initializing ρ using method of moments
ρhat = 1.155985356012513


In [4]:
gc = gcm.data[1]
β = gcm.β
ρ = gcm.ρ[1]
σ2 = gcm.σ2[1]

0.12991912209464118

### check each function within logl

In [5]:
using BenchmarkTools
@benchmark get_V!($ρ, $gc)

BenchmarkTools.Trial: 10000 samples with 972 evaluations.
 Range (min … max):  61.106 ns … 259.995 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     65.720 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   70.635 ns ±  17.424 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▆█▇▃    ▁▂▃         ▁                                       ▁
  █████▆▆▆▇████▇▇▇▆▆▆▇▇█▇█▇▆▇▆▆▇▆▆▆▆▆▆▅▆▅▅▅▆▆▆▆▅▆▅▆▅▆▆▄▅▅▄▄▅▄▄ █
  61.1 ns       Histogram: log(frequency) by time       155 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [6]:
@benchmark get_∇V!($ρ, $gc)

BenchmarkTools.Trial: 10000 samples with 982 evaluations.
 Range (min … max):  61.621 ns … 233.370 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     64.397 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   69.878 ns ±  17.256 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂█▅▅▅▄▁ ▂                                                    ▁
  █████████▆▆▇█▇▆▆▄▃▆▆▅▅▆▆▇▇▅▆▇▇▆▆▅▅▅▆▆▅▆▅▅▅▄▅▅▆▅▅▅▅▅▅▅▅▄▄▅▅▃▅ █
  61.6 ns       Histogram: log(frequency) by time       158 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [7]:
@benchmark get_∇2V!($ρ, $gc)

BenchmarkTools.Trial: 10000 samples with 972 evaluations.
 Range (min … max):  59.952 ns … 216.530 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     64.289 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   69.253 ns ±  13.735 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▆█▇▅▄▄▄▂  ▂▄▅▄                                              ▂
  ██████████▇█████▇▆▇▆▅▆███▇██▇▅▇▆▆▆▆▆▆▆▅▆▅▅▅▅▄▆▅▃▃▅▅▆▄▄▅▅▅▄▄▄ █
  60 ns         Histogram: log(frequency) by time       138 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [8]:
@benchmark update_res!($gc, $β)

BenchmarkTools.Trial: 10000 samples with 921 evaluations.
 Range (min … max):  111.625 ns … 406.170 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     116.896 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   123.171 ns ±  23.813 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅███▃▁       ▁▁                                               ▂
  ███████▆▅▅▇██████▇▆▆▇▇▇▆▇▆█▇▆▇█▆▇▇▇▇▇▇▆▇▆▅▅▅▅▅▇▆▆▆▆▆▅▄▄▄▄▅▆▄▆ █
  112 ns        Histogram: log(frequency) by time        237 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [9]:
@benchmark standardize_res!($gc)

BenchmarkTools.Trial: 10000 samples with 997 evaluations.
 Range (min … max):  127.147 ns … 412.124 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     127.836 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   135.597 ns ±  25.745 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅          ▁                                                 ▁
  ██▇▅▅▄▆▆▅▆▆██▅▆▆▇▇▇▇▇▇▅▆▆▅█▆▆█▆▅▆▅▆▆▆▆▅▆▄▄▅▄▆▄▅▅▅▅▃▄▅▄▄▅▅▄▃▅▅ █
  127 ns        Histogram: log(frequency) by time        264 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [10]:
@benchmark std_res_differential!($gc)

BenchmarkTools.Trial: 10000 samples with 204 evaluations.
 Range (min … max):  379.397 ns …  1.404 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     380.098 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   398.935 ns ± 72.313 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁         ▁▂             ▁                                  ▁
  ██▇▄▆▆▃▆▆▁▇██▇▆▄▅▆▅▆▅▆▅▆█▅█▆▅█▆▆▇▄▅▇▅▅▅▃▅▅▆▄▆▄▄▅▅▄▄▄▄▅▄▄▅▄▅▅ █
  379 ns        Histogram: log(frequency) by time       783 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
@benchmark GLMCopula.component_loglikelihood($gc)

BenchmarkTools.Trial: 10000 samples with 937 evaluations.
 Range (min … max):  103.902 ns … 335.584 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     106.013 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   110.972 ns ±  16.885 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃█▄▃▅ ▂       ▂ ▂                                             ▁
  ███████▄▆▄▆▇▇▇█▇█▇▆▆▅▆▅▅▆▆▆▆▄▆▇▆▆█▇▃▆█▅▅▅▇▅▅▆▇▆▅▄▅▅▁▅▅▃▅▅▄▄▄▄ █
  104 ns        Histogram: log(frequency) by time        196 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [12]:
@benchmark GLMCopula.glm_gradient($gc)

BenchmarkTools.Trial: 10000 samples with 195 evaluations.
 Range (min … max):  482.862 ns …   1.586 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     496.815 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   538.649 ns ± 127.486 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▂    ▁▁ ▃        ▁ ▁                                        ▁
  ███▇▆▄▆████▇▇▆▇▇▇███████▇▇▇▇▆▆▆▆▆▇▇▇▅▇▇▆▆▆▆▆▅▅▅▆▆▆▅▅▄▅▅▅▆▅▆▄▅ █
  483 ns        Histogram: log(frequency) by time       1.13 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [13]:
@benchmark GLMCopula.glm_hessian($gc)

BenchmarkTools.Trial: 10000 samples with 832 evaluations.
 Range (min … max):  148.151 ns … 459.624 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     148.863 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   156.235 ns ±  24.927 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆ ▁           ▁                                              ▁
  ████▄█▅▄▇▅▆▅▇▆▆█▅▆▅▆▆▆▇▆▆▇▅▆▇▄▄█▅▅▇▅▆▅▅▅▅▅▄▅▅▅▅▄▅▅▃▃▅▅▄▄▄▄▅▅▅ █
  148 ns        Histogram: log(frequency) by time        284 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# with just the loglikelihood

In [14]:
@benchmark loglikelihood!($gc, $β, $ρ, $σ2, false, false)

BenchmarkTools.Trial: 10000 samples with 199 evaluations.
 Range (min … max):  428.794 ns …  1.337 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     430.302 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   451.251 ns ± 69.852 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄ ▃ ▄       ▁ ▂                                             ▁
  ██▆█▆█▆▆▅▃▄▆▆█▆█▅▅▃▄▅▃▄▅▄▆▅▄▆▆▄▅█▅▅▇▅▅▄▄▅▄▅▄▃▅▄▄▄▄▃▃▄▄▃▄▃▄▄▃ █
  429 ns        Histogram: log(frequency) by time       805 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# with logl and gradient

In [15]:
@benchmark loglikelihood!($gc, $β, $ρ, $σ2, true, false)

BenchmarkTools.Trial: 10000 samples with 149 evaluations.
 Range (min … max):  687.611 ns …   2.104 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     689.819 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   718.541 ns ± 108.030 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂ ▄▁         ▁ ▂                                             ▁
  ██████▄▆▃▅▆▆▆▅█▅█▆▄▆▄▅▅▆▅▅▆▇▆▆▆▆▆▅█▅▄▄█▄▅▅▆▅▆▆▆▅▃▄▁▆▅▅▅▄▄▅▄▆▅ █
  688 ns        Histogram: log(frequency) by time       1.26 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.